### Cálculo de cuartiles de lista de valores con lista de frecuencias asociadas. 

In [1]:
# Con map, filter y reduce: 

def custom_cuartiles (valores, frecuencias=None):
    
    """ Calcula los cuartiles de la distribución en la lista 'valores' con las frecuencias absolutas especificadas
    en la lista 'frecuencias'. Ajuste de punto medio cuando una frecuencia es igual a la asociada a un cuartil. 
    
    Herramientas: map y reduce. 
    
    ¿Para qué? Para los casos en los que las frecuencias no se determinan por el número de repeticiones en la 
    lista, sino por una lista adjunta de frecuencias de cada valor (np.quantile no es aplicable directamente). 
    """
        
    from functools import reduce
    
    # Si no hay frecuencias, se imputan unitarias: 
    if frecuencias==None:
        frecuencias=[1]*len(valores)
        
    # Las dos listas ordenadas de acuerdo con valores: 
    listas_ordenadas=sorted(zip(valores,frecuencias))
    
    # Prescinde de tuplas con frecuencia nula (interfieren en ajuste para frecuencias exactas):
    tuplas=list(filter(lambda tupla: tupla[1]!=0, listas_ordenadas))
    
    # Frecuencias absolutas ordenadas de acuerdo con valores: 
    fr_ord=list(map(lambda tupla: tupla[1], tuplas))
    
    # Frecuencia absoluta acumulada i-ésima (i=1,2,...,len(frecuencias)):
    fr_acum=list(map(lambda i: reduce(lambda x,y: x+y, fr_ord[:i+1]),range(len(fr_ord))))
    
    # Frecuencia total (suma de frecuencias absolutas): 
    fr_T=fr_acum[-1]
    
    # Frecuencias relativas acumuladas: 
    fr_r_acum=list(map(lambda fr: fr/fr_T, fr_acum))
    
    # Valores con frecuencias no nulas ordenados: 
    valores_ord=list(map(lambda tupla: tupla[0], tuplas))
    
    # Cuartiles (lista Q):
    Q=[valores_ord[0]] # q_0 == min (primer elemento)
    # q_1 a q_3:
    for frq in [0.25,0.5,0.75]: 
        try:
            # Si alguna frecuencia coincide exactamente con la correspondiente al cuartil: 
            indice_exacto=fr_r_acum.index(frq)
            # Ajuste punto medio: 
            Q.append((valores_ord[indice_exacto]+valores_ord[indice_exacto+1])/2)
        except:
            # Primer valor con frecuencia por encima de la correspondiente al cuartil: 
            primera_fr_por_encima=list(filter(lambda fr: fr>frq,fr_r_acum))[0]
            Q.append(valores_ord[fr_r_acum.index(primera_fr_por_encima)])
            
    Q.append(valores_ord[-1]) # q_4 == max (último elemento)
    
    return Q   # [q0, q1, q2, q3, q4]

In [2]:
# Con list comprehension: 

def custom_cuartiles_bis (valores, frecuencias=None):
    
    """ Calcula los cuartiles de la distribución en la lista 'valores' con las frecuencias absolutas especificadas
    en la lista 'frecuencias'. Ajuste de punto medio cuando una frecuencia es igual a la asociada a un cuartil. 
    
    Herramientas: list comprehension.  
    
    ¿Para qué? Para los casos en los que las frecuencias no se determinan por el número de repeticiones en la 
    lista, sino por una lista adjunta de frecuencias de cada valor (np.quantile no es aplicable directamente). 
    """
    
    # Si no hay frecuencias, se imputan unitarias: 
    if frecuencias==None:
        frecuencias=[1]*len(valores)
        
    # Las dos listas ordenadas de acuerdo con valores: 
    listas_ordenadas=sorted(zip(valores,frecuencias))
    
    # Prescinde de tuplas con frecuencia nula (interfieren en ajuste para frecuencias exactas):
    tuplas=[tupla for tupla in listas_ordenadas if tupla[1]!=0]
    
    # Frecuencias absolutas ordenadas de acuerdo con valores: 
    fr_ord=[tupla[1] for tupla in tuplas]
    
    
    # Frecuencia absoluta acumulada i-ésima (i=1,2,...,len(frecuencias)):    
    fr_acum=[sum(fr_ord[:i+1]) for i in range(len(fr_ord))]
    
    # Frecuencia total (suma de frecuencias absolutas): 
    fr_T=fr_acum[-1]
    
    # Frecuencias relativas acumuladas: 
    fr_r_acum=[fr/fr_T for fr in fr_acum]
    
    # Valores con frecuencias no nulas ordenados: 
    valores_ord=[valores for (valores, fr) in tuplas]
    
    # Cuartiles (lista Q):
    Q=[valores_ord[0]] # q_0 == min (primer elemento)
    # q_1 a q_3:
    for frq in [0.25,0.5,0.75]: 
        try:
            # Si alguna frecuencia coincide exactamente con la correspondiente al cuartil: 
            indice_exacto=fr_r_acum.index(frq)
            # Ajuste punto medio: 
            Q.append((valores_ord[indice_exacto]+valores_ord[indice_exacto+1])/2)
        except:
            # Primer valor con frecuencia por encima de la correspondiente al cuartil:
            primera_fr_por_encima=[fr for fr in fr_r_acum if fr>frq][0]
            Q.append(valores_ord[fr_r_acum.index(primera_fr_por_encima)])
            
    Q.append(valores_ord[-1]) # q_4 == max (último elemento)
    
    return Q # [q0, q1, q2, q3, q4]

In [3]:
# Ejemplo: 

v=[5,9,1,4,7,2] # lista de valores
fr=[3,2,1,0,1,6] # lista de frecuencias asociadas

custom_cuartiles(v,fr), custom_cuartiles_bis(v,fr) 

([1, 2, 2, 5, 9], [1, 2, 2, 5, 9])

### Cambio de formato de lista: de lista de valores con frecuencias dadas por repetición a lista de valores con lista de frecuencias absolutas asociadas. 

In [4]:
# Con map: 

def fr_compactas(valores):
    """ A partir de una lista de valores con o sin repeticiones, devuelve una lista ordenada de valores únicos y 
    otra lista de frecuencias absolutas asociadas a cada valor. 
    
    Permite adaptar una lista de valores al formato de valores y frecuencias que se emplean en la función 
    custom_cuartiles. 
    
    Herramientas: map. 
    """
    # Lista ordenada de valores únicos: 
    valores_unicos_ord=sorted(list(set(valores)))
    # Lista de frecuencias absolutas asociadas: 
    fr=list(map(lambda valor: valores.count(valor), valores_unicos_ord))
    
    return valores_unicos_ord, fr

In [5]:
# Con list_comprehension:

def fr_compactas_bis(valores):
    """ A partir de una lista de valores con o sin repeticiones, devuelve una lista ordenada de valores únicos y 
    otra lista de frecuencias absolutas asociadas a cada valor. 
    
    Permite adaptar una lista de valores al formato de valores y frecuencias que se emplean en la función 
    custom_cuartiles. 
    
    Herramientas: list comprehension.
    """
    # Lista ordenada de valores únicos: 
    valores_unicos_ord=sorted(list(set(valores)))
    # Lista de frecuencias absolutas asociadas: 
    fr=[valores.count(valor) for valor in valores_unicos_ord]
    
    return valores_unicos_ord, fr

In [6]:
# Ejemplo: 
example=[18,45,66,70,76,83,88,90,90,95,95,98]

fr_compactas(example)

([18, 45, 66, 70, 76, 83, 88, 90, 95, 98], [1, 1, 1, 1, 1, 1, 1, 2, 2, 1])

In [7]:
fr_compactas_bis(example)

([18, 45, 66, 70, 76, 83, 88, 90, 95, 98], [1, 1, 1, 1, 1, 1, 1, 2, 2, 1])

In [8]:
# Ejemplo: 
# Cálculo de cuartiles con np.quantile: 

import numpy as np
custom_cuartiles(fr_compactas(example)[0],fr_compactas(example)[1])\
                    ==np.quantile(example,[0,.25,.5,.75,1],interpolation='midpoint')

array([ True,  True,  True,  True,  True])

In [9]:
custom_cuartiles_bis(fr_compactas(example)[0],fr_compactas(example)[1])\
                    ==np.quantile(example,[0,.25,.5,.75,1],interpolation='midpoint')

array([ True,  True,  True,  True,  True])

### Cambio de formato de lista: de lista de valores con lista de frecuencias absolutas asociadas a lista de valores con frecuencias establecidas por repetición

Permite transformar un par de listas (valores y frecuencias) en otra en el formato válido para np.quantile. 

In [10]:
# con map: 

def lista_unica(valores, fr):
    """ Transforma una lista de valores únicos y otra de frecuencias absolutas asociadas en una lista en la que 
    cada valor aparece tantas veces como corresponde a su frecuencia. 
    """
    
    from functools import reduce
    
    out=list(reduce(lambda L1, L2: L1+L2, map(lambda tupla: [tupla[0]]*tupla[1], zip(valores,fr))))
    
    return out

In [11]:
# con double list comprehension: 

def lista_unica_bis(valores, fr):
    """ Transforma una lista de valores únicos y otra de frecuencias absolutas asociadas en una lista en la que 
    cada valor aparece tantas veces como corresponde a su frecuencia. 
    """
    out=[valor for (valor,fr) in zip(valores,fr) for valor in [valor]*fr]
    
    return out

In [12]:
# Ejemplo: 

v=[5,9,1,4,7,2] # lista de valores
fr=[3,2,1,0,1,6] # lista de frecuencias asociadas

lista_unica(v,fr)

[5, 5, 5, 9, 9, 1, 7, 2, 2, 2, 2, 2, 2]

In [13]:
# Ejemplo: 

lista_unica_bis(v,fr)

[5, 5, 5, 9, 9, 1, 7, 2, 2, 2, 2, 2, 2]

In [14]:
# Ejemplo: 
lista_unica(fr_compactas(example)[0],fr_compactas(example)[1])==example

True

In [15]:
lista_unica_bis(fr_compactas(example)[0],fr_compactas(example)[1])==example

True

In [16]:
# Ejemplo/comprobación: 
np.quantile(lista_unica(v,fr),[0,0.25,0.5,0.75,1], interpolation='midpoint')==custom_cuartiles(v,fr)

array([ True,  True,  True,  True,  True])

In [17]:
np.quantile(lista_unica(v,fr),[0,0.25,0.5,0.75,1], interpolation='midpoint')==custom_cuartiles_bis(v,fr)

array([ True,  True,  True,  True,  True])